In [12]:
# The goal is to extract all ROUGE, METEOR, BLEU score in one csv

import pandas as pd

In [3]:
#ROUGE

import pyrouge
import logging
pyrouge_tools = '/home/ffajri/Workspace/pyrouge/tools/ROUGE-1.5.5'

def rouge_eval(ref_dir, dec_dir):
    """Evaluate the files in ref_dir and dec_dir with pyrouge, returning results_dict"""
    r = pyrouge.Rouge155(pyrouge_tools)
    r.model_filename_pattern = '#ID#'
    r.system_filename_pattern = '(\d+)'
    r.model_dir = ref_dir
    r.system_dir = dec_dir
    logging.getLogger('global').setLevel(
        logging.WARNING)  # silence pyrouge logging
    rouge_results = r.convert_and_evaluate()
    return r.output_to_dict(rouge_results)

In [27]:
import glob
import os
import shutil

#lang = 'RU_latin'
lang = 'FR'
model = 'PG'
MAIN = f'resulting_data/{lang}/'
gold_path = MAIN+'gold/'
pred_path = MAIN+f'pred_{model}/'

gold_temp = 'temp_gold'
pred_temp = 'temp_pred'

rouges = {}
for file in glob.glob(gold_path+'*'):
    fname = file.split('/')[-1]
    if os.path.exists(gold_temp):
        shutil.rmtree(gold_temp)
        shutil.rmtree(pred_temp)
    os.makedirs(gold_temp)
    os.makedirs(pred_temp)
    
    shutil.copy2(file, gold_temp)
    shutil.copy2(pred_path+fname, pred_temp)
    
    results_dict = rouge_eval(gold_temp, pred_temp)
    rouges[int(fname)] = results_dict
rouges = dict(sorted(rouges.items()))

!rm -r temp_gold temp_pred

In [28]:
#BLEU

from collections import Counter
from sacrebleu import BLEU

def read_file(p):
    lines = open(p, 'r').readlines()
    lines = [l.lower().strip() for l in lines]
    return [' '.join(lines)]

def compute_blue(idx):
    BLEU.NGRAM_ORDER=idx
    def extract_ngrams(line, min_order=1, max_order=BLEU.NGRAM_ORDER) -> Counter:
        """Extracts all the ngrams (min_order <= n <= max_order) from a sequence of tokens.
        :param line: A segment containing a sequence of words.
        :param min_order: Minimum n-gram length (default: 1).
        :param max_order: Maximum n-gram length (default: NGRAM_ORDER).
        :return: a dictionary containing ngrams and counts
        """
        ngrams = Counter()
        tokens = line.split()
        for n in range(min_order, max_order + 1):
            for i in range(0, len(tokens) - n + 1):
                ngram = ' '.join(tokens[i: i + n])
                ngrams[ngram] += 1
        return ngrams
    
    BLEU.extract_ngrams = extract_ngrams
    import sacrebleu
    bleu_scores = {}
    
    for file in glob.glob(gold_path+'*'):
        fname = file.split('/')[-1]
        p = read_file(pred_path+fname)
        g = read_file(file)
        bleu_scores[int(fname)] = sacrebleu.corpus_bleu(p, [g]).score
    
    return bleu_scores

bleu1 = compute_blue(1)
bleu2 = compute_blue(2)
bleu3 = compute_blue(3)
bleu4 = compute_blue(4)
bleu1 = dict(sorted(bleu1.items()))
bleu2 = dict(sorted(bleu2.items()))
bleu3 = dict(sorted(bleu3.items()))
bleu4 = dict(sorted(bleu4.items()))

In [29]:
#METEOR

ps=[]; gs=[]; ids=[]
for file in glob.glob(gold_path+'*'):
    fname = file.split('/')[-1]
    ids.append(int(fname))
    p = read_file(pred_path+fname)
    g = read_file(file)
    ps.append(p[0])
    gs.append(g[0])

f1 = open ('meteor_gold', 'w')
f2 = open ('meteor_pred', 'w')
for idx, d in enumerate (gs):     
    f1.write(gs[idx]+'\n')
    f2.write(ps[idx]+'\n')
f1.close()
f2.close()

!java -Xmx2G -jar /home/ffajri/Workspace/meteor/meteor-*.jar meteor_pred meteor_gold -l fr > meteor_output.txt
!rm meteor_gold
!rm meteor_pred

In [30]:
meteors = {}
i = 0
for line in open('meteor_output.txt', 'r').readlines():
    if 'Segment' in line:
        score = float(line.strip().split('\t')[-1])
        meteors[ids[i]] = score
        i+=1

In [31]:
def save_to_file():
    save_to = MAIN+f'{model}_traditional.csv'
    list_score = list(rouges.values())
    list_key = list(rouges.keys())
    df = pd.DataFrame()
    df['id'] = list_key
    df['ROUGE-1-F'] = [rouges[k]['rouge_1_f_score'] for k in list_key]
    df['ROUGE-2-F'] = [rouges[k]['rouge_2_f_score'] for k in list_key]
    df['ROUGE-3-F'] = [rouges[k]['rouge_3_f_score'] for k in list_key]
    df['ROUGE-L-F'] = [rouges[k]['rouge_l_f_score'] for k in list_key]
    df['ROUGE-S-F'] = [rouges[k]['rouge_s*_f_score'] for k in list_key]
    df['ROUGE-SU-F'] = [rouges[k]['rouge_su*_f_score'] for k in list_key]
    df['ROUGE-W-F'] = [rouges[k]['rouge_w_1.2_f_score'] for k in list_key]
    
    df['ROUGE-1-P'] = [rouges[k]['rouge_1_precision'] for k in list_key]
    df['ROUGE-2-P'] = [rouges[k]['rouge_2_precision'] for k in list_key]
    df['ROUGE-3-P'] = [rouges[k]['rouge_3_precision'] for k in list_key]
    df['ROUGE-L-P'] = [rouges[k]['rouge_l_precision'] for k in list_key]
    df['ROUGE-S-P'] = [rouges[k]['rouge_s*_precision'] for k in list_key]
    df['ROUGE-SU-P'] = [rouges[k]['rouge_su*_precision'] for k in list_key]
    df['ROUGE-W-P'] = [rouges[k]['rouge_w_1.2_precision'] for k in list_key]
    
    df['ROUGE-1-R'] = [rouges[k]['rouge_1_recall'] for k in list_key]
    df['ROUGE-2-R'] = [rouges[k]['rouge_2_recall'] for k in list_key]
    df['ROUGE-3-R'] = [rouges[k]['rouge_3_recall'] for k in list_key]
    df['ROUGE-L-R'] = [rouges[k]['rouge_l_recall'] for k in list_key]
    df['ROUGE-S-R'] = [rouges[k]['rouge_s*_recall'] for k in list_key]
    df['ROUGE-SU-R'] = [rouges[k]['rouge_su*_recall'] for k in list_key]
    df['ROUGE-W-R'] = [rouges[k]['rouge_w_1.2_recall'] for k in list_key]
    
    df['METEOR'] = [meteors[k] for k in list_key]
    df['BLEU-1'] = [bleu1[k] for k in list_key]
    df['BLEU-2'] = [bleu2[k] for k in list_key]
    df['BLEU-3'] = [bleu3[k] for k in list_key]
    df['BLEU-4'] = [bleu4[k] for k in list_key]
    df.to_csv(save_to, index=False)

In [32]:
save_to_file()